# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 7 2022 9:25AM,238968,10,SOTAH0000697,"Nextsource Biotechnology, LLC",Released
1,Jun 7 2022 9:18AM,238965,12,HH-34938,Hush Hush,Released
2,Jun 7 2022 9:18AM,238965,12,HH-34939,Hush Hush,Released
3,Jun 7 2022 9:18AM,238965,12,HH-34941,Hush Hush,Released
4,Jun 7 2022 9:18AM,238965,12,HH-34943,Hush Hush,Released
5,Jun 7 2022 9:18AM,238965,12,HH-34944,Hush Hush,Released
6,Jun 7 2022 9:18AM,238965,12,HH-34945,Hush Hush,Released
7,Jun 7 2022 9:18AM,238965,12,HH-34947,Hush Hush,Released
8,Jun 7 2022 9:18AM,238965,12,HH-34952,Hush Hush,Released
9,Jun 7 2022 9:18AM,238965,12,HH-34954,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,238964,Released,64
41,238965,Released,20
42,238966,Released,1
43,238967,Executing,1
44,238968,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
238964,NaN,64.0
238965,NaN,20.0
238966,NaN,1.0
238967,1.0,NaN
238968,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
238857,1.0,0.0
238868,0.0,1.0
238873,0.0,1.0
238879,6.0,38.0
238883,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
238857,1,0
238868,0,1
238873,0,1
238879,6,38
238883,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,238857,1,0
1,238868,0,1
2,238873,0,1
3,238879,6,38
4,238883,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,238857,1,
1,238868,,1
2,238873,,1
3,238879,6,38
4,238883,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 7 2022 9:25AM,238968,10,"Nextsource Biotechnology, LLC"
1,Jun 7 2022 9:18AM,238965,12,Hush Hush
21,Jun 7 2022 9:17AM,238967,12,Hush Hush
22,Jun 7 2022 9:17AM,238966,10,"Citieffe, Inc."
23,Jun 7 2022 9:13AM,238964,10,ISDIN Corporation
87,Jun 7 2022 9:09AM,238955,10,ISDIN Corporation
122,Jun 7 2022 9:08AM,238962,18,Innova Softgel LLC
123,Jun 7 2022 9:02AM,238958,20,"Exact-Rx, Inc."
130,Jun 7 2022 8:59AM,238957,10,Beach Products Inc
131,Jun 7 2022 8:59AM,238956,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 7 2022 9:25AM,238968,10,"Nextsource Biotechnology, LLC",,1
1,Jun 7 2022 9:18AM,238965,12,Hush Hush,,20
2,Jun 7 2022 9:17AM,238967,12,Hush Hush,1,
3,Jun 7 2022 9:17AM,238966,10,"Citieffe, Inc.",,1
4,Jun 7 2022 9:13AM,238964,10,ISDIN Corporation,,64
5,Jun 7 2022 9:09AM,238955,10,ISDIN Corporation,,35
6,Jun 7 2022 9:08AM,238962,18,Innova Softgel LLC,,1
7,Jun 7 2022 9:02AM,238958,20,"Exact-Rx, Inc.",,7
8,Jun 7 2022 8:59AM,238957,10,Beach Products Inc,,1
9,Jun 7 2022 8:59AM,238956,20,Alumier Labs Inc.,,22


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 7 2022 9:25AM,238968,10,"Nextsource Biotechnology, LLC",1,
1,Jun 7 2022 9:18AM,238965,12,Hush Hush,20,
2,Jun 7 2022 9:17AM,238967,12,Hush Hush,,1
3,Jun 7 2022 9:17AM,238966,10,"Citieffe, Inc.",1,
4,Jun 7 2022 9:13AM,238964,10,ISDIN Corporation,64,
5,Jun 7 2022 9:09AM,238955,10,ISDIN Corporation,35,
6,Jun 7 2022 9:08AM,238962,18,Innova Softgel LLC,1,
7,Jun 7 2022 9:02AM,238958,20,"Exact-Rx, Inc.",7,
8,Jun 7 2022 8:59AM,238957,10,Beach Products Inc,1,
9,Jun 7 2022 8:59AM,238956,20,Alumier Labs Inc.,22,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 7 2022 9:25AM,238968,10,"Nextsource Biotechnology, LLC",1,
1,Jun 7 2022 9:18AM,238965,12,Hush Hush,20,
2,Jun 7 2022 9:17AM,238967,12,Hush Hush,,1
3,Jun 7 2022 9:17AM,238966,10,"Citieffe, Inc.",1,
4,Jun 7 2022 9:13AM,238964,10,ISDIN Corporation,64,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 7 2022 9:25AM,238968,10,"Nextsource Biotechnology, LLC",1.0,NaN
1,Jun 7 2022 9:18AM,238965,12,Hush Hush,20.0,NaN
2,Jun 7 2022 9:17AM,238967,12,Hush Hush,NaN,1.0
3,Jun 7 2022 9:17AM,238966,10,"Citieffe, Inc.",1.0,NaN
4,Jun 7 2022 9:13AM,238964,10,ISDIN Corporation,64.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 7 2022 9:25AM,238968,10,"Nextsource Biotechnology, LLC",1.0,0.0
1,Jun 7 2022 9:18AM,238965,12,Hush Hush,20.0,0.0
2,Jun 7 2022 9:17AM,238967,12,Hush Hush,0.0,1.0
3,Jun 7 2022 9:17AM,238966,10,"Citieffe, Inc.",1.0,0.0
4,Jun 7 2022 9:13AM,238964,10,ISDIN Corporation,64.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2867328,169.0,2.0
12,477932,20.0,1.0
16,477814,6.0,20.0
17,477784,2.0,0.0
18,716733,3.0,0.0
19,1672379,9.0,16.0
20,1911299,106.0,41.0
21,716783,2.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2867328,169.0,2.0
1,12,477932,20.0,1.0
2,16,477814,6.0,20.0
3,17,477784,2.0,0.0
4,18,716733,3.0,0.0
5,19,1672379,9.0,16.0
6,20,1911299,106.0,41.0
7,21,716783,2.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,169.0,2.0
1,12,20.0,1.0
2,16,6.0,20.0
3,17,2.0,0.0
4,18,3.0,0.0
5,19,9.0,16.0
6,20,106.0,41.0
7,21,2.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,169.0
1,12,Released,20.0
2,16,Released,6.0
3,17,Released,2.0
4,18,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,18,19,20,21
Status,,,,,,,,
Executing,2.0,1.0,20.0,0.0,0.0,16.0,41.0,1.0
Released,169.0,20.0,6.0,2.0,3.0,9.0,106.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,18,19,20,21
0,Executing,2.0,1.0,20.0,0.0,0.0,16.0,41.0,1.0
1,Released,169.0,20.0,6.0,2.0,3.0,9.0,106.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,18,19,20,21
0,Executing,2.0,1.0,20.0,0.0,0.0,16.0,41.0,1.0
1,Released,169.0,20.0,6.0,2.0,3.0,9.0,106.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()